<a href="https://colab.research.google.com/github/omarsagoo/herpeton/blob/Carrie/notebook_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Colab Friendly Notebook

In [1]:
#pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 133.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.1

In [ ]:
#@title Environment Setup
import hashlib
import io
import os
import sys
from collections import Counter, defaultdict
from dataclasses import dataclass
from typing import Dict, Iterable, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, UnidentifiedImageError
from tqdm.auto import tqdm

# Install third-party packages if missing (Colab-friendly)
try:
    from datasets import load_dataset
except ImportError:
    !pip -q install datasets
    from datasets import load_dataset

try:
    import imagehash
except ImportError:
    !pip -q install ImageHash
    import imagehash

In [ ]:
#@title Mount Google Drive
CURATE_TO_DRIVE = True

if CURATE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_OUT = "/content/drive/MyDrive/herpeton"
else:
    BASE_OUT = "/content/herpeton"

os.makedirs(BASE_OUT, exist_ok=True)

# Controls to keep runtime reasonable
MAX_RECORDS = 15000          # hard cap across the stream (increase if you have time/compute)
SAMPLE_PER_SPECIES = 120     # images per species to collect/curate/EDA
REPORT_DIR = os.path.join(BASE_OUT, "_reports")
os.makedirs(REPORT_DIR, exist_ok=True)

print(f"Output base: {BASE_OUT}")
print(f"Reports   : {REPORT_DIR}")
print(f"Limits    : MAX_RECORDS={MAX_RECORDS}, SAMPLE_PER_SPECIES={SAMPLE_PER_SPECIES}")

In [ ]:
python quick_start.py

## Step 1: Extract Reptilia Metadata from BioTrove

Using HuggingFace's efficient filter() method to extract only Reptilia samples from the 135M sample dataset.

In [ ]:
def extract_reptilia_efficient():
    """
    Efficiently extract Reptilia samples using HuggingFace datasets with filtering.
    Since Reptilia exists in the dataset but is distributed throughout 135M samples,
    the dataset's built-in filtering capabilities are used.
    """
    from datasets import load_dataset
    from tqdm.auto import tqdm

    print("EFFICIENT REPTILIA EXTRACTION")
    print("="*60)
    print("Using filter() to extract only Reptilia class")


    # Load dataset in streaming mode
    print("Loading BioTrove-Train dataset...")
    dataset = load_dataset('BGLab/BioTrove-Train', streaming=True, split='train')

    # Filter for Reptilia class
    print("Applying Reptilia filter...")
    reptilia_dataset = dataset.filter(lambda example: example['class'] == 'Reptilia')

    print("Extracting Reptilia samples...")
    reptilia_samples = []

    pbar = tqdm(desc="Collecting Reptilia", unit=" samples")

    # Collect samples
    target_samples = 2000  # Collect 2000 Reptilia samples

    for idx, item in enumerate(reptilia_dataset):
        reptilia_samples.append(item)

        if idx < 10:  # Show first 10
            print(f"\n   Sample {idx+1}:")
            print(f"      scientificName: {item.get('scientificName', 'Unknown')}")
            print(f"      family: {item.get('family', 'Unknown')}")
            print(f"      order: {item.get('order', 'Unknown')}")

        pbar.update(1)
        pbar.set_description(f"Collected {len(reptilia_samples)} Reptilia")

        if len(reptilia_samples) >= target_samples:
            print(f"\n   Reached target of {target_samples} samples")
            break

    pbar.close()

    print(f"\n{'='*60}")
    print(f"EXTRACTION COMPLETE:")
    print(f"   Total Reptilia samples collected: {len(reptilia_samples)}")

    if reptilia_samples:
        # Save to parquet chunks
        os.makedirs("biotrove_metadata", exist_ok=True)
        chunk_size = 250
        saved_files = []

        print(f"\n   Saving {len(reptilia_samples)} samples in chunks of {chunk_size}...")

        for i in range(0, len(reptilia_samples), chunk_size):
            chunk_data = reptilia_samples[i:i+chunk_size]
            chunk_df = pd.DataFrame(chunk_data)

            chunk_filename = f"biotrove_metadata/reptilia_chunk_{i//chunk_size}.parquet"
            chunk_df.to_parquet(chunk_filename, index=False)
            saved_files.append(chunk_filename)

            print(f"      Chunk {i//chunk_size}: {len(chunk_data)} samples -> {chunk_filename}")

        print(f"\n   ✓ Saved {len(saved_files)} parquet files")

        # Show species diversity
        species_list = [s.get('scientificName', 'Unknown') for s in reptilia_samples]
        unique_species = set(species_list)
        print(f"   ✓ {len(unique_species)} unique species")

        # Show top families
        families = [s.get('family', 'Unknown') for s in reptilia_samples]
        from collections import Counter
        top_families = Counter(families).most_common(10)
        print(f"\n   Top 10 families:")
        for family, count in top_families:
            print(f"      {family}: {count} samples")

        return {
            'samples': reptilia_samples,
            'saved_files': saved_files,
            'unique_species': len(unique_species)
        }
    else:
        print("   ✗ No Reptilia samples found")
        return None

# Run the efficient extraction
reptilia_data = extract_reptilia_efficient()

## Step 2: Verify Extracted Data

Check the extracted Reptilia samples before processing.

In [ ]:
# Verify the extracted Reptilia data
import glob

print("VERIFICATION: Extracted Reptilia Data")
print("="*60)

# Find all reptilia chunk files
chunk_files = sorted(glob.glob("biotrove_metadata/reptilia_chunk_*.parquet"))

print(f"Found {len(chunk_files)} parquet chunk files:")
for file in chunk_files:
    df = pd.read_parquet(file)
    print(f"   {file}: {len(df)} samples")

# Load all chunks
print("\nLoading all Reptilia data...")
all_dfs = [pd.read_parquet(f) for f in chunk_files]
reptilia_df = pd.concat(all_dfs, ignore_index=True)

print(f"\nTOTAL REPTILIA DATASET:")
print(f"   Total samples: {len(reptilia_df):,}")
print(f"   Unique species: {reptilia_df['scientificName'].nunique()}")
print(f"   Unique families: {reptilia_df['family'].nunique()}")
print(f"   Unique orders: {reptilia_df['order'].nunique()}")

print(f"\nCOLUMNS:")
print(f"   {list(reptilia_df.columns)}")

print(f"\nORDER DISTRIBUTION:")
order_counts = reptilia_df['order'].value_counts()
for order, count in order_counts.items():
    print(f"   {order}: {count} samples ({count/len(reptilia_df)*100:.1f}%)")

print(f"\nTOP 15 SPECIES:")
species_counts = reptilia_df['scientificName'].value_counts().head(15)
for species, count in species_counts.items():
    family = reptilia_df[reptilia_df['scientificName'] == species]['family'].iloc[0]
    print(f"   {species:<40} ({family:<20}): {count} samples")

print(f"\nFIRST 3 SAMPLE ROWS:")
print(reptilia_df.head(3).to_string(index=False))

print(f"\n{'='*60}")
print("✓ Reptilia dataset successfully extracted and verified!")
print("✓ Ready for BioTrove processing pipeline")

VERIFICATION: Extracted Reptilia Data
Found 0 parquet chunk files:

Loading all Reptilia data...


NameError: name 'pd' is not defined

## Step 3: Download Images and Create Dataset

Download images from URLs and create ML-ready dataset with image-text pairs.

In [ ]:
# Install arbor_process library for BioTrove processing
import subprocess
import sys

try:
    import arbor_process
    print("arbor_process is already installed")
except ImportError:
    print("Installing arbor_process library...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "arbor-process"])
    import arbor_process
    print("Successfully installed arbor_process")

In [ ]:
# Download Reptilia images from metadata URLs
import os
import glob
import requests
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed

print("REPTILIA IMAGE DOWNLOAD")
print("="*60)

# Create output directories
os.makedirs("biotrove_processed", exist_ok=True)
os.makedirs("biotrove_processed/images", exist_ok=True)

# Load all Reptilia chunks
chunk_files = sorted(glob.glob("biotrove_metadata/reptilia_chunk_*.parquet"))
print(f"Loading {len(chunk_files)} Reptilia parquet chunks...")

all_chunks = [pd.read_parquet(f) for f in chunk_files]
reptilia_metadata = pd.concat(all_chunks, ignore_index=True)

print(f"Total Reptilia samples: {len(reptilia_metadata):,}")
print(f"Unique species: {reptilia_metadata['scientificName'].nunique()}")

# Save combined metadata
metadata_path = "biotrove_processed/reptilia_metadata.csv"
reptilia_metadata.to_csv(metadata_path, index=False)
print(f"\nSaved combined metadata to: {metadata_path}")

def download_image(row):
    """Download a single image from URL"""
    photo_id = row['photo_id']
    """Download single image from URL"""

    try:
        response = requests.get(photo_url, timeout=10)
        response.raise_for_status()

        # Open and verify image
        img = Image.open(BytesIO(response.content))

        # Save image
        img_filename = f"biotrove_processed/images/{photo_id}.jpg"
        img.convert('RGB').save(img_filename, 'JPEG', quality=95)

        return {'photo_id': photo_id, 'status': 'success', 'path': img_filename}
    except Exception as e:
        return {'photo_id': photo_id, 'status': 'failed', 'error': str(e)}

# Download images with parallel processing
print("\n" + "="*60)
print("DOWNLOADING IMAGES")
print("="*60)
print(f"Downloading {len(reptilia_metadata)} images with 4 parallel workers...")

successful_downloads = []
failed_downloads = []

with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit all download tasks
    futures = {executor.submit(download_image, row): idx
               for idx, row in reptilia_metadata.iterrows()}

    # Process completed downloads with progress bar
    with tqdm(total=len(futures), desc="Downloading") as pbar:
        for future in as_completed(futures):
            result = future.result()

            if result['status'] == 'success':
                successful_downloads.append(result)
            else:
                failed_downloads.append(result)

            pbar.update(1)
            pbar.set_description(f"Downloaded: {len(successful_downloads)}, Failed: {len(failed_downloads)}")

print(f"\n{'='*60}")
print(f"DOWNLOAD COMPLETE!")
print(f"  Successfully downloaded: {len(successful_downloads)} images")
print(f"  Failed downloads: {len(failed_downloads)} images")
print(f"  Success rate: {len(successful_downloads)/len(reptilia_metadata)*100:.1f}%")

# Create image-text pairs dataset
print("\n" + "="*60)
print("CREATING IMAGE-TEXT PAIRS")
print("="*60)

successful_photo_ids = [d['photo_id'] for d in successful_downloads]
successful_metadata = reptilia_metadata[reptilia_metadata['photo_id'].isin(successful_photo_ids)].copy()

# Add image paths
successful_metadata['image_path'] = successful_metadata['photo_id'].apply(
    lambda x: f"biotrove_processed/images/{x}.jpg"
)

# Create text descriptions
successful_metadata['text_description'] = successful_metadata.apply(
    lambda row: f"{row['scientificName']} ({row['common_name']}) - {row['family']}, Order {row['order']}, Class Reptilia",
    axis=1
)

# Save final dataset
final_dataset_path = "biotrove_processed/reptilia_dataset_final.csv"
successful_metadata.to_csv(final_dataset_path, index=False)

print(f"Created {len(successful_metadata)} image-text pairs")
print(f"Saved to: {final_dataset_path}")

print(f"\n{'='*60}")
print(f"PROCESSING COMPLETE!")
print(f"  Metadata: {metadata_path}")
print(f"  Images: biotrove_processed/images/ ({len(successful_downloads)} files)")
print(f"  Final dataset: {final_dataset_path}")
print(f"  Dataset ready for computer vision training!")

In [ ]:
# Explore the processed dataset
import random

print("PROCESSED DATASET EXPLORATION")
print("="*60)

# Check directory structure
print("\nDirectory structure:")
for root, dirs, files in os.walk("biotrove_processed"):
    level = root.replace("biotrove_processed", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files)-5} more files")

# Load processed metadata
if os.path.exists("biotrove_processed/metadata.csv"):
    processed_df = pd.read_csv("biotrove_processed/metadata.csv")

    print(f"\n{'='*60}")
    print(f"PROCESSED DATASET STATISTICS:")
    print(f"  Total samples: {len(processed_df):,}")
    print(f"  Unique species: {processed_df['scientificName'].nunique()}")
    print(f"  Unique families: {processed_df['family'].nunique()}")
    print(f"  Unique orders: {processed_df['order'].nunique()}")

    print(f"\nOrder distribution:")
    for order, count in processed_df['order'].value_counts().items():
        print(f"  {order}: {count} samples ({count/len(processed_df)*100:.1f}%)")

    print(f"\nTop 10 species:")
    for species, count in processed_df['scientificName'].value_counts().head(10).items():
        print(f"  {species}: {count} samples")

    # Show sample images if available
    image_dir = "biotrove_processed/images"
    if os.path.exists(image_dir):
        image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        if image_files:
            print(f"\n{'='*60}")
            print(f"Sample images available: {len(image_files)}")

            # Display a few random samples
            sample_images = random.sample(image_files, min(3, len(image_files)))
            print(f"\nDisplaying {len(sample_images)} random samples...")

            for img_file in sample_images:
                img_path = os.path.join(image_dir, img_file)
                # Get metadata for this image
                img_id = os.path.splitext(img_file)[0]
                if img_id.isdigit():
                    sample_info = processed_df[processed_df['photo_id'] == int(img_id)]
                    if not sample_info.empty:
                        print(f"\n  Image: {img_file}")
                        print(f"    Species: {sample_info.iloc[0]['scientificName']}")
                        print(f"    Family: {sample_info.iloc[0]['family']}")
                        print(f"    Common name: {sample_info.iloc[0].get('common_name', 'N/A')}")

print(f"\n{'='*60}")
print("Dataset ready for computer vision training.")